In [1]:

def is_square_free( f ) :
    """
    This function checks if the polynomial `f` is square-free.
    """
    return gcd( f, f.derivative() ) == 1
    
def rad( f ) :
    """
    This function computes the radical of `f`.
    """
    # find the ring of polynomials
    P = f.parent()
    # return the radical
    return P(f/gcd(f,f.derivative()))
    
def square_free_decomposition( f ) :
    """
    This function takes a polynomial f and returns its square-free decomposition.
    """
    # The base ring must be a field of characteristic zero
    assert f.base_ring().characteristic() == 0, "The characteristic must be zero!"
    assert f.base_ring().is_field(), "The base ring must be a field!"

    # The base ring must be a field of characteristic zero
    assert f.base_ring().characteristic() == 0, "The characteristic must be zero!"
    assert f.base_ring().is_field(), "The base ring must be a field!"

    lcf = f.leading_coefficient()
    G = list(f.monic().squarefree_decomposition())
    return Factorization( G, unit = lcf, sort = false )
    
def depress( f ) :
    """
    This function computes the depressed form of a given polynomial `f`.
    """
    # take the leading coefficient
    lcf = f.leading_coefficient()
    # normalize f
    f /= lcf
    # the degree of f
    n = f.degree()
    # depress f
    g = f( x-f[n-1]/n )
    return g
    
def Cauchy_1( f ) :
    """
    Given a polynomial `f`, this function computes `M` s.t. all the roots of `f` are contained in `(-M,M)`.
    It uses the "first" Cauchy bound.
    """
    n = f.degree()
    return 1 + max([abs(f[j]/f[n]) for j in [0..n-1]])
    
def Cauchy_2( f ) :
    """
    Given a polynomial `f`, this function computes `M` s.t. all the roots of `f` are contained in `(-M,M)`.
    It uses the "second" Cauchy bound.
    """
    n = f.degree()
    return max( (n*abs(f[n-j]/f[n]))^(1/j) for j in [1..n] )
    
def Hong( f ) :
    """
    Given a polynomial `f`, thsi function computes `M > 0` s.t. all the REAL roots of `f` are contained in `(-M,M)`.
    It uses Hong's theorem.
    """
    if f.leading_coefficient() < 0 :
        f = -f
    n = f.degree()
    H = max( min( (-f[j]/f[i])^(1/(i-j)) for i in [j+1..n] if f[i] > 0 ) for j in [0..n-1] if f[j] < 0 )
    return 2*H
    
def local_max( f ) :
    """
    Given a polynomial `f`, thsi function computes `M > 0` s.t. all the REAL roots of `f` are contained in `(-M,M)`.
    It uses local-max algorithm.
    """
    n = f.degree()
    M = 0
    t = 1
    i_max = n
    for j in [n-1,n-2,..0] :
        if f[j] > f[i_max] :
            i_max = j
            t = 1
        if f[j] < 0 :
            m = ( -2^t*f[j]/f[i_max] )^( 1/(i_max - j) )
            t = t+1
            M = max(m, M)
    return M

<p><span style="color: #ff0000;">The above hidden cell will execute automatically upon opening the worksheet. It contains the following functions from exercise set 7:</span></p>
<ul>
<li><span style="color: #ff0000; font-family: courier new,courier;">is_square_free( f )</span></li>
<li><span style="color: #ff0000; font-family: courier new,courier;">rad( f )</span></li>
<li><span style="color: #ff0000; font-family: courier new,courier;">square_free_decomposition( f, omit_trivial_factors = true )</span></li>
<li><span style="font-family: courier new,courier; color: #ff0000;">depress( f )</span></li>
<li><span style="font-family: courier new,courier; color: #ff0000;">Cauchy_1( f )</span></li>
<li><span style="font-family: courier new,courier; color: #ff0000;">Cauchy_2( f )</span></li>
<li><span style="font-family: courier new,courier; color: #ff0000;">Hong( f )</span></li>
<li><span style="font-family: courier new,courier; color: #ff0000;">local_max( f )</span></li>
</ul>

<h2>Strurm sequence</h2>
<p><strong>Exercise:</strong> Compute the Sturm sequence of a polynomial $f = x^{4} - 5 x^{2} + 4$ <strong>step by step</strong>.</p>

In [2]:
# define a polynomial f
P.<x> = QQ[]
f = 4-5*x^2+x^4; show("f = " + latex(f))

f = x^{4} - 5 x^{2} + 4

In [3]:
# g_0 is just f
g0 = f; show("g_0 = " + latex(g0))

g_0 = x^{4} - 5 x^{2} + 4

In [4]:
# g_1 is the derivative of f
g1 = f.derivative(); show("g_1 = " + latex(g1))

g_1 = 4 x^{3} - 10 x

In [5]:
# next terms are minus remainders
g2 = -g0.mod(g1); show("g_2 = " + latex(g2))

g_2 = \frac{5}{2} x^{2} - 4

In [6]:
g3 = -g1.mod(g2); show("g_3 = " + latex(g3))

g_3 = \frac{18}{5} x

In [7]:
g4 = -g2.mod(g3); show("g_4 = " + latex(g4))

g_4 = 4

<p><strong>Exercise:</strong> Write a function that computes the Sturm sequence of a polynomial.</p>

In [8]:
def Sturm_sequence( f ) :
    """
    Given a polynomial `f` this function returns its Sturm sequence.
    """
    # g_0 is just f, while g_1 is the derivative of f
    g = f.derivative()
    # append both to the list
    G = [ f, g ]
    # repeat till yu hit a constant
    while not g.is_constant() :
        # substitute f by g and g by the minus reminder...
        f, g = g, -f.mod(g)
        # ...and append it to the list
        G.append(g)
    return G

In [9]:
# check with the previous example
show( Sturm_sequence(f) )

[x^4 - 5*x^2 + 4, 4*x^3 - 10*x, 5/2*x^2 - 4, 18/5*x, 4]

<p><strong>Exercise:</strong> Write a function that counts the number of essential sign changes in a list. Test it with $(1,3,-2,-4,5,17)$ and $(0,1,0,1,-1,0,0,-1,0,1,0,1)$.</p>

In [10]:
def sign_changes( L ) :
    """
    This function counts the number sign changes in `L`
    """
    # remove all zeros and substitute each element by its signature
    L = [ sgn(a) for a in L if a != 0 ]
    # leave only those element that differs from it predecessors
    S = [ L[j] for j in range(1, len(L)) if L[j] != L[j-1] ]
    # the number of sign changes in L equals the length of S
    return len(S)

In [11]:
sign_changes( [1,3,-2,-4,5,17] )

2

In [12]:
sign_changes( [0,1,0,1,-1,0,0,-1,0,1,0,1] )

2

In [13]:
# alternative solution - using an explicit loop

def sign_changes( L ) :
    """
    This function counts the number sign changes in `L`
    """
    # initialize the counter to zero
    counter = 0
    # the initial signature is the signature of the first NONZERO term
    for a in L :
        if a != 0 :
            s = sgn(a)
            break
    else :
        return 0
    # for every element of L
    for a in L :
        # if it has a different sign...
        if sgn(a)*s < 0 :
            # ...then remember this sign...
            s = sgn(a)
            # ...and increment the counter
            counter += 1
    return counter

<p><strong>Exercise:</strong> How many real roots does $f=x^{5} - 5 x^{3} + 4 x - 1$ have? How many of them are positive?</p>

In [14]:
# define a polynomial f
f = x^5-5*x^3+4*x-1; show("f=" + latex(f))

f= x^{5} - 5 x^{3} + 4 x - 1

In [15]:
# compute the Sturm sequence of f
S = Sturm_sequence( f ); show("S = " +latex(S))

S = \left[x^{5} - 5 x^{3} + 4 x - 1, 5 x^{4} - 15 x^{2} + 4, 2 x^{3} - \frac{16}{5} x + 1, 7 x^{2} + \frac{5}{2} x - 4, \frac{883}{490} x - \frac{29}{49}, \frac{1889881}{779689}\right]

In [16]:
# substitute $x = \pm\infty$. Warning Sage doesn't like an infinity to be substituted
# into a constant polynomial, hence we just copy the last term verbatim
Splus  = [ g(oo) for g in S[:-1] ] + [S[-1]]; show("S_+ = " + latex(Splus))
Sminus = [ g(-oo) for g in S[:-1] ] + [S[-1]]; show("S_- = " + latex(Sminus))

S_+ = \left[+\infty, +\infty, +\infty, +\infty, +\infty, \frac{1889881}{779689}\right]

S_- = \left[-\infty, +\infty, -\infty, +\infty, -\infty, \frac{1889881}{779689}\right]

In [17]:
# Sturm theorem asserts tham the number of sign changes in $S_-$ minus the
# number of sign changes in $S_+$  equals the number of real roots of $f$
sign_changes(Sminus) - sign_changes(Splus)

5

In [18]:
# substitute x = 0
S0 = [ g(0) for g in S ]; show("S_0 = " + latex(S0))

S_0 = \left[-1, 4, 1, -4, -\frac{29}{49}, \frac{1889881}{779689}\right]

In [19]:
# the number of real roots = sign changes in $S_0$ minus sign changes in $S_+$
sign_changes(S0) - sign_changes(Splus)

3

<p><strong>Exercise:</strong> Write a function that counts the number of roots $f$ siting in a real interval $(a,b)\subset \mathbb{R}\cup \{\pm\infty\}$.</p>

In [20]:
def count_roots( f, a=-oo, b=oo, S = [] ) :
    """
    This function counts the number of roots of `f` in the interval `(a,b)`.
    
    The optional parameter `S` is the Sturm sequence of `f`, if it has already been computed.
    """
    assert a < b, "Improper interval!"
    
    # if we don't have the Sturm sequence, then we must compute it
    if S == [] :
        S = Sturm_sequence( f )
    # substitute the endpoint of the interval to the Sturm sequence
    Sa = [ g(a) for g in S[:-1] ] + [S[-1]]
    Sb = [ g(b) for g in S[:-1] ] + [S[-1]]
    # return the difference of the numbers of sign changes
    return sign_changes(Sa) - sign_changes(Sb)

In [21]:
# check with the previous example
count_roots(x^5-5*x^3+4*x-1, -oo,oo)

5

<h2>Real root isolation<strong><br /></strong></h2>
<p><strong>Exercise:</strong> Write a function that isolates real roots of a given polynomial.</p>

In [77]:
def root_isolation( f, (a,b), S = [] ) :
    """
    This function takes the following arguments:
        
       - a polynomial `f`
       
       - an interval `(a,b)`
       
       - (optionally) the Sturm sequence of `f`
       
    as a result it return a list of pairs `( (a_j,b_j) : j ∊ {1,...,n} )` such that:
        
       - `f(a_j) = 0` iff `a_j = b_j`
       
       - `f` has precisely one root `x_j` s.t. `a_j < x_j < b_j`
       
    """
    assert a < b, "Improper interval!"
    
    # if a is infinite, then substitute it by a bound
    if a == -oo :
        a = -RDF( local_max(f) )
    # if b is infinite, then substitute it by a bound
    if b == oo :
        b = RDF( local_max(f) )
        
    # the variable of our polynomial
    x = f.parent().0

    # if any of the endpoints is a root of f, then append it to the list
    # and narrow the interval
    if f(a) == 0 :
        epsilon = RDF( 1/local_max(f(x+a).list()[::-1]) )
        return [(a,a)] + root_isolation(f, (a+epsilon,b), S)
    if f(b) == 0 :
        epsilon = RDF( 1/local_max(f(x+b).list()[::-1]) )
        return root_isolation(f, (a,b-epsilon), S) + [(b,b)]
        
    # compute the Sturm sequence if it is not provided
    if S == [] :
        S = Sturm_sequence( f )
    # count the number of root of f in (a,b)
    p = count_roots(f, (a,b), S )
    # if f has no roots, then return an empty list
    if p == 0 :
        return []
    # if f has just one root then we are done
    if p == 1 :
        return [(a,b)]
    # otherwise bisect the interval and call the function recursively for each half
    if p > 1 :
        c = (a+b)/2
        # this "if" protects us again c being counted twice
        if f(c) != 0 :
            return root_isolation(f, (a,c), S) + root_isolation(f, (c,b), S)
        else :
            epsilon = RDF(1/local_max(f(x-c).list()[::-1]))
            return root_isolation(f, (a,c-epsilon), S) + [(c,c)] + root_isolation(f, (c+epsilon,b), S)

In [79]:
# check with the previous example
f = x^5-5*x^3+4*x-1
L = root_isolation(f, (-oo,oo)); L

[(-3.1622776601683795, -1.5811388300841898),
 (-1.5811388300841898, 0.0),
 (0.0, 0.7905694150420949),
 (0.7905694150420949, 1.5811388300841898),
 (1.5811388300841898, 3.1622776601683795)]

In [80]:
for (a,b) in L :
    if a < b :
        f.plot( a,b, figsize = 4 )

<p><strong>Exercise: </strong>Improve the solution of the previous exercise, so that neither $f'$ nor $f"$ has a root in any of the return intervals.</p>

In [81]:
def root_isolation( f, (a,b), S = [] ) :
    """
    This function takes the following arguments:
        
       - a polynomial `f`
       
       - an interval `(a,b)`
       
       - (optionally) the Sturm sequence of `f`
       
    as a result it return a list of pairs `( (a_j,b_j) : j ∊ {1,...,n} )` such that:
        
       - `f(a_j) = 0` iff `a_j = b_j`
       
       - `f` has precisely one root `x_j` s.t. `a_j < x_j < b_j`
       
       - `f'`, `f"` have not roots in `(a_j, b_j)` for every `j`
       
    """
    assert a < b, "Improper interval!"

    # if a is infinite, then substitute it by a bound
    if a == -oo :
        a = -RDF( local_max(f) )
    # if b is infinite, then substitute it by a bound
    if b == oo :
        b = RDF( local_max(f) )
        
    # the variable of our polynomial
    x = f.parent().0

    # if any of the endpoints is a root of f, then append it to the list
    # and narrow the interval
    if f(a) == 0 :
        epsilon = RDF( 1/local_max(f(x+a).list()[::-1]) )
        return [(a,a)] + root_isolation(f, (a+epsilon,b), S)
    if f(b) == 0 :
        epsilon = RDF( 1/local_max(f(x+b).list()[::-1]) )
        return root_isolation(f, (a,b-epsilon), S) + [(b,b)]
        
    # compute the Sturm sequence if it is not provided
    if S == [] :
        S = Sturm_sequence( f )
    # count the number of root of f in (a,b)
    p = count_roots(f, (a,b), S )
    # if f has no roots, then return an empty list
    if p == 0 :
        return []

    # if `f` has just one root, then count the number of roots of its derivatives
    if p == 1 :
        fp = f.derivative()
        fpp = f.derivative(2)
        pp = count_roots(fp, (a,b))
        ppp = count_roots(fpp, (a,b))
        # if they bot do not have roots, then we are done
        if pp == 0 and ppp == 0 :
            return [(a,b)]
            
    # otherwise bisect the interval and call the function recursively for each half
    c = (a+b)/2
    # this "if" protects us again c being counted twice
    if f(c) != 0 :
        L = root_isolation(f, (a,c), S)
        U = root_isolation(f, (c,b), S)
        return L + U
    else :
        epsilon = RDF(1/local_max(f(x-c).list()[::-1]))
        return root_isolation(f, (a,c-epsilon), S) + [(c,c)] + root_isolation(f, (c+epsilon,b), S)

In [83]:
# check with the previous example
f = x^5-5*x^3+4*x-1
L = root_isolation(f, (-oo,oo)); L

[(-1.9764235376052373, -1.7787811838447136),
 (-1.1858541225631423, -0.7905694150420949),
 (0.0, 0.39528470752104744),
 (0.7905694150420949, 1.1858541225631423),
 (1.9764235376052373, 2.3717082451262845)]

In [84]:
for (a,b) in L :
    if a < b :
        f.plot( a,b, figsize = 4 )